<a href="https://colab.research.google.com/github/map72ru/biz_learn/blob/main/%D0%9A%D0%A0_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606616 sha256=e8ef8362cb844ad1428fb7e8c5c5683414c1edd0da862a966c2d895592cecb0f
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [2]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/hw2')

import sys
sys.path.append('/content/hw2/MyDrive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/hw2


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [4]:
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(recommended_list, bought_list)

    if sum(flags) == 0:
        return 0

    sum_ = 0
    for i in range(1, k + 1):

        if flags[i - 1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k

    result = sum_ / min(len(recommended_list), k)

    return result

def map_k(recommended_list, bought_list, k=5):
    assert len(recommended_list) == len(bought_list)

    sum = 0
    for recommended, bought in zip(recommended_list, bought_list):
        sum = sum + ap_k(recommended, bought, k)
        
    return sum/len(recommended_list)

In [5]:
data = pd.read_csv('/content/hw2/MyDrive/data/retail_train.csv')
item_features = pd.read_csv('/content/hw2/MyDrive/data/product.csv')
user_features = pd.read_csv('/content/hw2/MyDrive/data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=10000, popularity_level=0.5, max_price=100)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 10001


In [7]:
recommender = MainRecommender(data_train_lvl_1, n_factors=30, iterations=30)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/10001 [00:00<?, ?it/s]

In [10]:
train_users = data_train_lvl_1['user_id'].unique()
data_val_lvl_1 = data_val_lvl_1[data_val_lvl_1['user_id'].isin(train_users)]

Считаем baseline


In [11]:
result_bsl = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_bsl.columns=['user_id', 'actual']
result_bsl['candidates'] = result_bsl['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=5))
result_bsl.head(2)

,user_id,actual,candidates
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[896666, 1087895, 856942, 877391, 5577022]"
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1107760, 244960, 932631, 843418, 5567582]"


In [12]:
map5 = 100*map_k(result_bsl['candidates'], result_bsl['actual'], 5)
map5

10.475437780877073

In [13]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Посчитаем 75 рекомендаций для обучения модели второго уровня
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=75))

In [14]:
#Добавим расчетных данных для клиентов

def add_user_features(data_set, user_features):

  cols = data_set[['user_id', 'quantity', 'sales_value', 'day', 'basket_id']]
  newdf = cols.copy()
  # количество покупок и сумма всех покупок
  newdf1 = newdf.groupby('user_id').agg({'quantity': 'count', 'sales_value': 'sum'})
  # Средний чек
  newdf1['check']=newdf1['sales_value']/newdf1['quantity']

  newdf2 = newdf.groupby('user_id').agg({'day': 'count', 'quantity': 'sum', 'basket_id': 'count'})
  # среднее количество покупок за раз
  newdf2['avg_qnt_day'] = newdf2['quantity']/newdf2['day']
  # частота покупок
  newdf2['frq'] = newdf2['day']/len(data_train_lvl_1.day.unique())
  # Среднее количество товара в корзине
  newdf2['bsk_qnt'] = newdf2['quantity']//newdf2['basket_id']
  newdf = newdf2.merge(newdf1, how='left', on='user_id')

  cols = newdf[['avg_qnt_day', 'frq', 'sales_value', 'check', 'bsk_qnt']]
  newdf = cols.copy()

  if len(user_features.columns.intersection(newdf.columns)) > 0:
    for user_id, row in newdf.iterrows():
      user_features.loc[user_features['user_id'] == user_id, 'frq'] = row['frq']
      user_features.loc[user_features['user_id'] == user_id, 'user_sales_value'] = row['sales_value']
      user_features.loc[user_features['user_id'] == user_id, 'check'] = row['check']
      user_features.loc[user_features['user_id'] == user_id, 'chavg_qnt_dayeck'] = row['avg_qnt_day']
      user_features.loc[user_features['user_id'] == user_id, 'bsk_qnt'] = row['bsk_qnt']
    return user_features

  return user_features.merge(newdf, how='left', on='user_id')

In [15]:
user_features_ext = add_user_features(data_train_lvl_2, user_features)
user_features_ext = add_user_features(data_val_lvl_2, user_features_ext)

user_features_ext.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,avg_qnt_day,frq,sales_value,check,bsk_qnt,user_sales_value,chavg_qnt_dayeck
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,1.112782,0.138280,341.78,2.440488,1.0,200.12,1.134146
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,1.288889,0.220911,187.65,2.653130,1.0,347.56,1.259542


In [16]:
# Добавим расчетные данные для товаров
def add_items_features(data_set, item_features):
  cols = data_set[['item_id', 'quantity', 'sales_value', 'week_no']]
  newdf = cols.copy()

# Сколько всего продали такого товара и на какую сумму
  newdf1 = newdf.groupby('item_id').agg({'quantity': 'sum', 'sales_value': 'sum'})
  newdf2 = newdf.groupby('item_id').agg({'week_no': 'count', 'quantity':'sum'})
# Среднее количество товаров, продаваемое в неделю
  newdf2['avg_qnt_week'] = newdf2['week_no']/newdf2['quantity']
  newdf2.drop(['quantity', 'week_no'], axis=1, inplace=True);
  newdf2 = newdf1.merge(newdf2, how='left', on='item_id')

  if len(item_features.columns.intersection(newdf2.columns)) > 0:
    for item_id, row in newdf2.iterrows():
        item_features.loc[item_features['item_id'] == item_id, 'item_quantity'] = row['quantity']
        item_features.loc[item_features['item_id'] == item_id, 'item_sales_value'] = row['sales_value']
        item_features.loc[item_features['item_id'] == item_id, 'avg_qnt_week'] = row['avg_qnt_week']
    return item_features

  return item_features.merge(newdf2, how='left', on='item_id')

In [17]:
item_features_ext = add_items_features(data_train_lvl_2, item_features)
item_features_ext = add_items_features(data_val_lvl_2, item_features_ext)

item_features_ext.query('item_id in @data_val_lvl_2.item_id').head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,quantity,sales_value,avg_qnt_week,item_quantity,item_sales_value
107,29512,69,GROCERY,Private,CANNED JUICES,TOMATO JUICE (OVER 50% JCE),46 OZ,NaN,NaN,1.0,1.0,0.99
131,30356,69,DRUG GM,Private,KITCHEN GADGETS,GADGETS/TOOLS,,NaN,NaN,1.0,1.0,2.39


In [18]:
feat1 = data_train_lvl_2.merge(item_features, how="inner", on="item_id")[['item_id', 'user_id', 'quantity', 'department', 'week_no']]
feat = feat1.copy()
departments = feat.groupby('department').agg({'quantity': 'sum'})
departments = departments.sort_values('quantity', ascending=False)[:10]
departments

,quantity
department,
KIOSK-GAS,15267213
MISC SALES TRAN,2840719
GROCERY,147466
DRUG GM,23792
PRODUCE,18139
MEAT-PCKGD,8455
MEAT,7336
DELI,4131
PASTRY,3200


In [19]:
# Расчет фичей покупатель/товар
# Кол-во покупок юзером конкретной категории в неделю.
# 
train_lvl_2 = data_train_lvl_2.copy()
feat1 = data_train_lvl_2.merge(item_features, how="inner", on="item_id")[['item_id', 'user_id', 'quantity', 'department', 'week_no']]
feat = feat1.copy()
departments = feat.groupby('department').agg({'quantity': 'sum'})
departments = departments.sort_values('quantity', ascending=False)[:10]
for department in departments.index:
  print(f'train:{department}')
  col_name = department.replace(' ', '').replace('&', '').replace('/', '').replace('.', '').replace('-', '')
  if col_name == '':
    col_name = 'EMPTY'
  dep_only=feat.query('department == @department')
  feat1 = dep_only.groupby(['item_id', 'user_id', 'week_no']).agg({'quantity': 'sum'})
  feat2 = dep_only.groupby(['item_id']).agg({'week_no': 'count', 'quantity': 'sum'})
  feat1.rename(columns = {'quantity': 'avg_'+col_name}, inplace=True)
# Среднее кол-во покупок всеми юзерами конкретной категории в неделю
  feat2['avg_cnt_' + col_name] = feat2['quantity']/feat2['week_no']

  train_lvl_2 = train_lvl_2.merge(feat1, how="left", on=['item_id','user_id','week_no'])
  for index, row in feat2.iterrows():
# (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю). Категория MEAT
    train_lvl_2['avg_by_week_' + col_name] = train_lvl_2.loc[train_lvl_2['item_id'] == index, 'avg_' + col_name]/row['avg_cnt_' + col_name]
    train_lvl_2.loc[train_lvl_2['item_id'] == index, 'avg_cnt_'+ col_name] = row['avg_cnt_'+ col_name]


val_lvl_2 = data_val_lvl_2.copy()
feat1 = data_val_lvl_2.merge(item_features, how="inner", on="item_id")[['item_id', 'user_id', 'quantity', 'department', 'week_no']]
feat = feat1.copy()
departments = feat1.groupby('department').agg({'quantity': 'sum'})
departments = departments.sort_values('quantity', ascending=False)[:10]
for department in departments.index:
  print(f'test:{department}')
  col_name = department.replace(' ', '').replace('&', '').replace('/', '').replace('.', '').replace('-', '')
  if col_name == '':
    col_name = 'EMPTY'
  dep_only=feat.query('department == @department')
  feat1 = dep_only.groupby(['item_id', 'user_id', 'week_no']).agg({'quantity': 'sum'})
  feat2 = dep_only.groupby(['item_id']).agg({'week_no': 'count', 'quantity': 'sum'})
  feat1.rename(columns = {'quantity': 'avg_'+col_name}, inplace=True)
# Среднее кол-во покупок всеми юзерами конкретной категории в неделю
  feat2['avg_cnt_' + col_name] = feat2['quantity']/feat2['week_no']

  val_lvl_2 = val_lvl_2.merge(feat1, how="left", on=['item_id','user_id','week_no'])
  for index, row in feat2.iterrows():
# (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю). Категория MEAT
    val_lvl_2['avg_by_week_' + col_name] = val_lvl_2.loc[val_lvl_2['item_id'] == index, 'avg_' + col_name]/row['avg_cnt_' + col_name]
    val_lvl_2.loc[val_lvl_2['item_id'] == index, 'avg_cnt_'+ col_name] = row['avg_cnt_'+ col_name]

train:KIOSK-GAS
train:MISC SALES TRAN
train:GROCERY
train:DRUG GM
train:PRODUCE
train:MEAT-PCKGD
train:MEAT
train:DELI
train:PASTRY
train:NUTRITION
test:KIOSK-GAS
test:MISC SALES TRAN
test:GROCERY
test:DRUG GM
test:PRODUCE
test:MEAT-PCKGD
test:MEAT
test:DELI
test:PASTRY
test:NUTRITION


In [20]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,2070,1105426,1
0,2070,6442443,1
0,2070,879194,1
0,2070,12263816,1


In [21]:
#targets_lvl_2 = train_lvl_2[['user_id', 'item_id', 'avg_meat', 'avg_meat_by_week', 'avg_cnt']].copy()
targets_lvl_2 = train_lvl_2.copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [22]:
targets_lvl_2 = targets_lvl_2.merge(item_features_ext, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features_ext, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity_x,sales_value_x,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,avg_KIOSKGAS,avg_by_week_KIOSKGAS,avg_cnt_KIOSKGAS,avg_MISCSALESTRAN,avg_by_week_MISCSALESTRAN,avg_cnt_MISCSALESTRAN,avg_GROCERY,avg_by_week_GROCERY,avg_cnt_GROCERY,avg_DRUGGM,avg_by_week_DRUGGM,avg_cnt_DRUGGM,avg_PRODUCE,avg_by_week_PRODUCE,avg_cnt_PRODUCE,avg_MEATPCKGD,avg_by_week_MEATPCKGD,avg_cnt_MEATPCKGD,avg_MEAT,avg_by_week_MEAT,avg_cnt_MEAT,avg_DELI,avg_by_week_DELI,avg_cnt_DELI,avg_PASTRY,avg_by_week_PASTRY,avg_cnt_PASTRY,avg_NUTRITION,avg_by_week_NUTRITION,avg_cnt_NUTRITION,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,quantity_y,sales_value_y,avg_qnt_week,item_quantity,item_sales_value,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,avg_qnt_day,frq,sales_value,check,bsk_qnt,user_sales_value,chavg_qnt_dayeck
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,5.0,19.95,0.8,5.0,17.96,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,86.029412,0.706577,617.29,2.928401,26.0,1227.0,26.00716
1,2070,6442443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,378,GROCERY,National,BAG SNACKS,POTATO CHIPS,10 OZ,6.0,15.00,1.0,5.0,12.50,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,86.029412,0.706577,617.29,2.928401,26.0,1227.0,26.00716


In [23]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [24]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [25]:
# Модель второго уровня
from lightgbm import LGBMClassifier

In [26]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict(X_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


In [27]:
#add_featues_lvl_2 = val_lvl_2[['user_id', 'item_id', 'avg_meat', 'avg_meat_by_week', 'avg_cnt']].copy()
add_featues_lvl_2 = val_lvl_2.copy()

In [28]:
users_lvl_2_val = pd.DataFrame(val_lvl_2['user_id'].unique())
users_lvl_2_val.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2_val = users_lvl_2_val[users_lvl_2_val['user_id'].isin(train_users)]

users_lvl_2_val['candidates'] = users_lvl_2_val['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))

In [29]:
s = users_lvl_2_val.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2_val = users_lvl_2_val.drop('candidates', axis=1).join(s)

users_lvl_2_val.head(4)

,user_id,item_id
0,338,876626
0,338,13002975
0,338,938165
0,338,939860


In [30]:
users_lvl_2_val_ext = users_lvl_2_val.merge(add_featues_lvl_2, how="left", on=['user_id', "item_id"])

In [31]:
pred_2 = users_lvl_2_val_ext.merge(user_features_ext, how="left", on="user_id")
pred_2 = pred_2.merge(item_features_ext, how="left", on="item_id")

pred_2.head(2)

,user_id,item_id,basket_id,day,quantity_x,sales_value_x,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,avg_KIOSKGAS,avg_by_week_KIOSKGAS,avg_cnt_KIOSKGAS,avg_MISCSALESTRAN,avg_by_week_MISCSALESTRAN,avg_cnt_MISCSALESTRAN,avg_GROCERY,avg_by_week_GROCERY,avg_cnt_GROCERY,avg_DRUGGM,avg_by_week_DRUGGM,avg_cnt_DRUGGM,avg_PRODUCE,avg_by_week_PRODUCE,avg_cnt_PRODUCE,avg_MEATPCKGD,avg_by_week_MEATPCKGD,avg_cnt_MEATPCKGD,avg_MEAT,avg_by_week_MEAT,avg_cnt_MEAT,avg_DELI,avg_by_week_DELI,avg_cnt_DELI,avg_PASTRY,avg_by_week_PASTRY,avg_cnt_PASTRY,avg_NUTRITION,avg_by_week_NUTRITION,avg_cnt_NUTRITION,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,avg_qnt_day,frq,sales_value_y,check,bsk_qnt,user_sales_value,chavg_qnt_dayeck,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,quantity_y,sales_value,avg_qnt_week,item_quantity,item_sales_value
0,338,876626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2382,GROCERY,National,DOMESTIC WINE,PREMIUM 750ML WINES,750 ML,9.0,80.91,0.333333,NaN,NaN
1,338,13002975,4.156028e+10,656.0,1.0,3.04,339.0,-0.61,2023.0,94.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.428571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2843,MEAT,National,BEEF,RIBS,,9.0,35.78,0.700000,10.0,38.58


In [32]:
X_val = pred_2
cat_feats = X_val.columns[2:].tolist()
X_val[cat_feats] = X_val[cat_feats].astype('category')

In [33]:
pred_2['proba'] = lgb.predict_proba(X_val)[:,1]

In [34]:
recomendations = pred_2[["user_id", "item_id", "proba"]]

In [35]:
lgbm_val = pd.DataFrame(recomendations.sort_values(['user_id', 'proba'], ascending=False).groupby('user_id').apply(lambda x: x['item_id'].iloc[:5].values)).rename(columns={0:'lgbm'})

In [36]:
val_actual = pd.DataFrame(data_val_lvl_2.groupby('user_id')['item_id'].unique()).rename(columns={'item_id': 'actual'}).reset_index()

In [37]:
total_val = lgbm_val.merge(val_actual, how='left', on='user_id')

In [38]:
total_val.head(2)

,user_id,lgbm,actual
0,1,"[9677939, 8293439, 8293439, 8091601, 856942]","[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[1092937, 13842217, 1083856, 1085778, 1084551]","[835476, 851057, 872021, 878302, 879948, 90963..."


Посчитаем метрику для валидационной выборки

In [39]:
map5 = 100*map_k(total_val['lgbm'], total_val['actual'])
map5

36.74787581699343

In [40]:
def transform_data_for_eval(dataset, rec_col, user_col='user_id'):
    '''
    Func for transforming recommendations into kaggle evaluation format

    Parameters:
    dataset (pd.DataFrame): Dataset with 2 required columns:
        rec_col - column with recommendations should be iterable
        user_col - columns with user id

    rec_col (str): name of column in dataset with recommendations

    user_col (str): name of column in dataset with user id

    Returns:
    pd.DataFrame: DataFrame in suitable format

   '''
    eval_dataset = dataset[[user_col, rec_col]].copy()
    eval_dataset[rec_col] = eval_dataset[rec_col].apply(lambda x: ' '.join([str(i) for i in x]))
    eval_dataset.rename(columns={
        user_col: 'UserId',
        rec_col: 'Predicted'
    }, inplace=True)
    return eval_dataset

In [ ]:
out.to_csv('/content/hw2/MyDrive/data/result.csv', index = False)

In [41]:
test_users = pd.read_csv('/content/hw2/MyDrive/data/test_users.csv')

In [42]:
userids = test_users.user_id.to_list()

In [44]:
data_test = data.query('user_id in @userids')

In [45]:
feat1 = data_test.merge(item_features, how="inner", on="item_id")[['item_id', 'user_id', 'quantity', 'department', 'week_no']]
feat = feat1.copy()
departments = feat1.groupby('department').agg({'quantity': 'sum'})
departments = departments.sort_values('quantity', ascending=False)[:10]
for department in departments.index:
  print(f'test:{department}')
  col_name = department.replace(' ', '').replace('&', '').replace('/', '').replace('.', '').replace('-', '')
  if col_name == '':
    col_name = 'EMPTY'
  dep_only=feat.query('department == @department')
  feat1 = dep_only.groupby(['item_id', 'user_id', 'week_no']).agg({'quantity': 'sum'})
  feat2 = dep_only.groupby(['item_id']).agg({'week_no': 'count', 'quantity': 'sum'})
  feat1.rename(columns = {'quantity': 'avg_'+col_name}, inplace=True)
# Среднее кол-во покупок всеми юзерами конкретной категории в неделю
  feat2['avg_cnt_' + col_name] = feat2['quantity']/feat2['week_no']

  data_test = data_test.merge(feat1, how="left", on=['item_id','user_id','week_no'])
  for index, row in feat2.iterrows():
# (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю). Категория MEAT
    data_test['avg_by_week_' + col_name] = data_test.loc[data_test['item_id'] == index, 'avg_' + col_name]/row['avg_cnt_' + col_name]
    data_test.loc[data_test['item_id'] == index, 'avg_cnt_'+ col_name] = row['avg_cnt_'+ col_name]

test:KIOSK-GAS
test:MISC SALES TRAN
test:GROCERY
test:DRUG GM
test:PRODUCE
test:MEAT-PCKGD
test:MEAT
test:DELI
test:PASTRY
test:NUTRITION


In [49]:
pred_test = data_test.merge(user_features_ext, how="left", on="user_id")
pred_test = pred_test.merge(item_features_ext, how="left", on="item_id")

In [50]:
X_test = pred_test
cat_feats = X_test.columns[2:].tolist()
X_test[cat_feats] = X_test[cat_feats].astype('category')

In [51]:
pred_test = lgb.predict_proba(X_test)[:,1]

In [52]:
pred_result = pd.DataFrame()
pred_result['proba'] = pred_test
pred_result['item_id'] = X_test['item_id']
pred_result['user_id'] = X_test['user_id']

In [53]:
lgbm_val = pd.DataFrame(pred_result.sort_values(['user_id', 'proba'], ascending=False).groupby('user_id').apply(lambda x: x['item_id'].iloc[:5].values)).rename(columns={0:'lgbm'})

In [54]:
lgbm_val

,lgbm
user_id,
1,"[940947, 977658, 956274, 1095374, 854496] Cate..."
2,"[1001266, 854852, 1077555, 1098066, 846833] Ca..."
3,"[910032, 910032, 1096727, 910032, 846823] Cate..."
6,"[840361, 851494, 851903, 863447, 927291] Categ..."
7,"[865569, 886703, 995436, 1028321, 1029743] Cat..."
...,...
2494,"[914585, 893867, 862276, 889509, 947201] Categ..."
2496,"[999270, 1080472, 1042907, 5588161, 951821] Ca..."
2498,"[952806, 6633273, 6633224, 1110031, 978077] Ca..."


In [68]:
result_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1707 entries, 1 to 2500
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lgbm    1707 non-null   object
dtypes: object(1)
memory usage: 106.7+ KB


In [64]:
userids = lgbm_val.index

In [67]:
result_data = lgbm_val.copy()

In [70]:
out = transform_data_for_eval(result_data, 'lgbm', user_col='user')

In [76]:
out.to_csv('/content/hw2/MyDrive/data/result.csv', columns=['UserId', 'Predicted'], index=False)